# 필터링
- 영상의 필터링(image filtering)
    - 영상에서 필요한 정보만 통과시키고 원치 않는 정보는 걸러내는 작업

- 주파수 공간에서의 필터링(Frequency domain filtering)
    - 퓨리에 트랜스폼를 이용한 변환
    - 저주파(영상의 부드러운 부분), 고주파(날카로운 부분, 픽셀값이 급격히 바뀌는 부분) 부분으로 변환하는 방법
- 공간적 필터링 (spatial domain filtering)
    - 영상의 픽셀 값을 직접 이용하는 필터링 방법
        - 대상 좌표의 픽셀 값과 주변 픽셀 값을 동시에 사용
    - 주로 마스크(mask) 연산을 이용함 (마스크=커널(kernel)=윈도우(window)=템플릿(template))

# 필터링 : 마스크 연산
- 다양한 모양과 크기의 마스크
- 마스크의 형태와 값에 따라 필터의 역할이 결정됨
    - 영상 부드럽게 만들기
    - 영상 날카롭게 만들기
    - 에지(edge) 검출
    - 잡음 제거
- CNN의 Correlation (Convolution) 연산하고 방식이 같음
    - 원래는 Correlation이 맞는말인데 관습적으로 Convolution이라 씀
    - CNN은 최외각 픽셀 처리할 때 제로패딩을 하는데 opencv는 마지막을 기준으로 대칭으로 처리한다.
    
- OpenCV 필터링에서 지원하는 가장자리 픽셀 확장 방법
| BorderTypes 열거형 상수 | 설명 |
| :-- | :-- |
| BORDER_CONSTANT | 외각을 0으로 채움(zero-padding) |
| BORDER_REPLICATE | 최외각 픽셀값으로 채움 |
| BORDER_REFLECT | 최외각을 기준으로 최외각 픽셀부터 그 다음 픽셀들로 대칭처리함 |
| BORDER_REFLECT_101 | 최외각을 기준으로 그 다음 픽셀들로 대칭처리함 |
| BORDER_REFLECT101 | BORDER_REFLECT_101과 같음 |
| BORDER_DEFAULT | BORDER_REFLECT_101과 같음 |

# OpenCV API

## 기본적인 2D 필터링
```python
cv2.filter2D(src, ddepth, kernel, dst=None, anchor=None, delta=None, borderType=None)
```

- src : 입력 영상
- ddepth : 출력 영상 데이터 타입 (e.g) cv2.CV_8U, cv2.CV_32F, cv2.CV_64F, 
    - -1을 지정하면 src와 같은 타입의 dst 영상으 생성
- kerenl : 필터 마스크 행렬, (실수형)
- anchor : 고정점 위치. (-1, -1) 이면 필터 중앙을 고정점으로 사용
- delta : 추가적으로 더할 값
- borderType : 가장자리 픽셀 확장 방식
- dst : 출력 영상

# 블러링
- 영상을 부드럽게 만들어준다.

## 평균 값 필터(mean filter)
- 영상의 특정 좌표 값을 주변 픽셀 값들의 산술 평균으로 설정
- 픽셀들 간의 그레이스케일 값 변화가 줄어들어 날카로운 에지가 무뎌지고, 영상에 있는 잡음의 영향이 사라지는 효과
- 필터 크기가 커질수록 연산량이 많아져서 성능저하 있을 수 있음
- 평균값의 필터는 커질수록 필터링의 화질이나 성능이 떨어짐을 볼 수 있다.

**평균 값 필터링 함수**
```python
cv2.blur(src, ksize, dst=None, anchor=None, borderType=None)
```
- src : 입력 영상
- ksize : 평균값 필터 크기. (width, height) 형태의 튜플
- dst : 결과 영상. 입력 영상과 같은 크기 & 같은 타입

In [1]:
import sys
import numpy as np
import cv2

src = cv2.imread('data/ch04/rose.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()
    

# 평균값 필터 생성 3x3
kernel = np.array([[1/9, 1/9, 1/9],
                             [1/9, 1/9, 1/9],
                             [1/9, 1/9, 1/9]], dtype=np.float32)

kernel2 = np.ones((3,3), np.float32) / 9

cv2.imshow('kernel', cv2.filter2D(src, -1, kernel))
cv2.imshow('kernel2', cv2.filter2D(src, -1, kernel))

cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()

In [2]:
src = cv2.imread('data/ch04/rose.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()


dst = cv2.blur(src, (3,3))
cv2.imshow('dst', dst)

cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
src = cv2.imread('data/ch04/rose.bmp', cv2.IMREAD_GRAYSCALE)

for ksize in (3, 5, 7):
    dst = cv2.blur(src, (ksize, ksize))
    desc = 'Mean : {} x {}'.format(ksize, ksize)
    cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 1, cv2.LINE_AA)
    
    cv2.imshow('dst', dst)
    cv2.waitKey()
    
cv2.destroyAllWindows()

## 가우시안 필터
- 평균값 필터에 의한 블러링의 단점
    - 필터링 대상 위치에서 가까이 있는 픽셀과 멀리 있는 픽셀이 모두 같은 가중치를 사용하여 평균을 계싼
    - 멀리 있는 픽셀의 영향을 많이 받을 수 있음
<br>
<br>
    
- 가우시안 필터
    - 가까운 픽셀은 큰 가중치를, 멀리 있는 픽셀은 작은 가중치를 사용하여 평균 계산
    - 1차원 가우시안 함수(Gaussian Function) - 정규분포
        - $G_{\mu, \sigma}(x) = \frac{1}{\sqrt{2\pi} \sigma} e^{-\frac{(x-\mu)^2}{2{\sigma}^2}}$
        - 좌우대칭
        - mean=median=mode
        - 68%(1$\sigma$), 95%(2$\sigma$), 99.7%(3$\sigma$)
    - 2차원 가우시안($\mu_x=\mu_y=0, \sigma_x=\sigma_y=\sigma$)
        - $ G_{\sigma})x,y) = \frac{1}{2\pi{\sigma}^2} e^{(-\frac{x^2+y^2}{2{\sigma}^2})}$
        - 2차원 가우시안 필터 마스크($\sigma=1.0$)
            - 필터 마스크 크기 : ($8\sigma+1$)(float) 또는 ($6\sigma + 1$)(uint)

가우시안 필터링 함수
```python
cv2.GaussianBlur(src, ksize, sigmaX, dst=None, sigmaY=None, borderType=None)
```
- src : 입력 영상. 각 채널 별로 처리됨
- dst : 출력 영상. src와 같은 크기, 같은 타입
- ksize : 가우시안 커널 크기. (0, 0)을 지정하면 sigma 값에 의해 자동 결정됨(강제값 안주는게 좋음->(0,0)주자)
- sigmaX : X방향 sigma
- sigmaY : Y방향 sigma. 0이면 sigmaX와 같에 설정
- borderType : 가장자리 픽셀 확장 방식

In [7]:
src = cv2.imread('data/ch04/rose.bmp', cv2.IMREAD_GRAYSCALE)

dst = cv2.GaussianBlur(src, (0,0), 2)
dst2 = cv2.blur(src, (7,7))

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.imshow('dst2', dst2)

cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
src = cv2.imread('data/ch04/rose.bmp', cv2.IMREAD_GRAYSCALE)

for sigmaX in (1,2,3):
    dst = cv2.GaussianBlur(src, (0,0), sigmaX)
    desc = f'sigma : {sigmaX}'
    cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 1, cv2.LINE_AA)
    
    cv2.imshow('dst', dst)
    cv2.waitKey()
    
cv2.destroyAllWindows()

# 샤프닝

## 언샤프 마스크 필터링 (Unsharp mask filtering)
- 날카롭지 않은(unsharp) 영상, 즉, 부드러워진 영상을 이용하여 날카로운 영상을 생성
- 엣지부분에서 contrast를 변화함
- 이건 opencv에 함수로 구현이 안되어 있음(직접만들어야한다.)

In [93]:
src = cv2.imread('data/ch04/rose.bmp', cv2.IMREAD_GRAYSCALE)

# 블러 한 영상
blr = cv2.GaussianBlur(src, (0,0), 2)
# 언샤프닝 2f - g
# dst = cv2.addWeighted(src, 2, blr, -1, 0)
dst = np.clip(2.0 *src - blr, 0, 255).astype(np.uint8)

cv2.imshow('dst', dst)
cv2.waitKey()    
cv2.destroyAllWindows()

언샤프 마스크 필터 구현하기
- 샤프닝 정도를 조절할 수 있도록 수식 변경
- $h(x,y) = f(x,y) + \alpha \cdot g(x,y)$
    - -> $h(x,y) = f(x,y) + \alpha\cdot (f(x,y) - \bar{f}(x,y)) = (1+\alpha)\cdot f(x,y) - \alpha \cdot \bar{f}(x,y)$
    - -> $h(x,y) = (1+\alpha)\cdot f(x,y) - \alpha \cdot G_{\sigma}(f(x,y))$

In [41]:
from tqdm import tqdm

In [70]:
src = cv2.imread('data/ch04/rose.bmp', cv2.IMREAD_GRAYSCALE)# .astype(np.float32)
blr = cv2.GaussianBlur(src, (0,0), 2)
for alpha in tqdm(range(1,5)):
    # float(alpha)*
    # 왜 차이가 나는거지??
    print(blr)
    dst2 = np.clip(np.float32((1+alpha)*src - blr), 0, 255).astype(np.uint8)
    dst3 = np.clip((float(1+alpha)*src - float(alpha)*blr), 0, 255).astype(np.uint8)
    cv2.imshow('dst2', dst2)
    cv2.imshow('dst3', dst3)
    dst4 = np.clip(np.float32((1+alpha)*src - (alpha)*blr), 0, 255).astype(np.uint8)
    cv2.imshow('dst4', dst4)
    cv2.waitKey()
cv2.destroyAllWindows()

  0%|          | 0/4 [00:00<?, ?it/s]

[[50 52 59 ... 25 24 23]
 [51 54 61 ... 26 24 24]
 [55 58 65 ... 27 26 25]
 ...
 [43 43 43 ... 54 53 53]
 [43 43 43 ... 50 50 50]
 [43 43 43 ... 48 48 48]]


 25%|██▌       | 1/4 [01:18<03:55, 78.48s/it]

[[50 52 59 ... 25 24 23]
 [51 54 61 ... 26 24 24]
 [55 58 65 ... 27 26 25]
 ...
 [43 43 43 ... 54 53 53]
 [43 43 43 ... 50 50 50]
 [43 43 43 ... 48 48 48]]


 50%|█████     | 2/4 [01:18<01:04, 32.48s/it]

[[50 52 59 ... 25 24 23]
 [51 54 61 ... 26 24 24]
 [55 58 65 ... 27 26 25]
 ...
 [43 43 43 ... 54 53 53]
 [43 43 43 ... 50 50 50]
 [43 43 43 ... 48 48 48]]


 75%|███████▌  | 3/4 [01:19<00:17, 17.79s/it]

[[50 52 59 ... 25 24 23]
 [51 54 61 ... 26 24 24]
 [55 58 65 ... 27 26 25]
 ...
 [43 43 43 ... 54 53 53]
 [43 43 43 ... 50 50 50]
 [43 43 43 ... 48 48 48]]


100%|██████████| 4/4 [01:19<00:00, 19.84s/it]


### 컬러 영상에 대한 언샤프 마스크 필터 구현

In [61]:
src = cv2.imread('data/ch04/rose.bmp')

src_ycrcb = cv2.cvtColor(src, cv2.COLOR_BGR2YCrCb)

# 밝기 성분만 조절
src_f = src_ycrcb[:, :, 0].astype(np.float32) # 여기서 float32으로 바꿈.
# blur 결과는 src의 타입과 같기때문에 uint로 넣으면 실수부가 다 짤림. 미세한 변화가 사라짐
blr = cv2.GaussianBlur(src_f, (0,0), 2)
src_ycrcb[:, :, 0] = np.clip(2. * src_f - blr, 0, 255).astype(np.uint8)

dst = cv2.cvtColor(src_ycrcb, cv2.COLOR_YCrCb2BGR)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# 잡음 제거

## 영상의 잡음
- 영상의 잡음(Noise)
    - 영상의 픽셀 값에 추가되는 원치 않는 형태의 신호
    - $ f(x,y) = s(x,y) + n(x,y) $
        - f : 획득된 영상
        - s : 원본 신호
        - n : 잡음
- 잡음의 종류 : 카메라, 센서의 노이즈, 아날로그-디지털 변환 사이의 잡음 등
    - 가우시안 잡음(Gaussian noise)
    - 소금&후추 잡음(Salt&Pepper) (요즘엔 보기 힘듦)

## 미디언 필터(Median filter)
- 주변 픽셀들의 값들을 정렬하여 그 중앙값(median)으로 픽셀 값을 대체
- 소금-후추 잡음 제거에 효과적(요즘엔 별로 안씀)

미디언 필터링 함수
```python
cv2.medianBlur(src, ksize, dst=None)
```

- src : 입력 영상. 각 채널 별로 처리됨
- ksize : 커널 크기. 1보다 큰 홀수를 지정
- dst : 출력 영상. src와 같은 크기, 같은 타입

In [83]:
src = cv2.imread('data/ch04/noise.bmp', cv2.IMREAD_GRAYSCALE)

dst = cv2.medianBlur(src, 3)
cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 양방향 필터

### 가우시안 필터
- 가우시안 잡음 제거에는 가우시안 필터가 효과적
- 너무 하면 엣지가 모호해짐

### 양방향 필터(Bilateral filter)
- 에지 보전 잡음 제거 필터(edge-preserving noise removal filter)의 하나
- 평균 값 필터 또는 가우시안 필터는 에지 부근에서도 픽셀 값을 평탄하게 만드는 단점이 있음
- 기준 픽셀과 이웃 픽셀과의 거리, 그리고 픽셀 값의 차이를 함께 고려하여 블러링 정도를 조절

$$ BF[I]_p = \frac{1}{W_p} \sum_{q\in S} G_{\sigma_s} (\vert\vert p -q \vert\vert) G_{\sigma_r}(\vert I_p - I_q \vert)I_q$$
- p, q : 픽셀의 좌표값
- I : 이미지의 픽셀값
- 픽셀의 가우시안 함수 값과 픽셀좌표의 p2놈 가우시안 함의 결합 꼴

양방향 필터링 함수
```python
cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace, dst=None, borderType=None)
```
- src : 입력 영상, 8비트 또는 실수형, 1채널 또는 3채널
- d : 필터링에 사용될 이웃 픽셀의 거리(지름). 음수(-1)를 입력하면 sigmaSpace 값에 의해 자동 결정됨
- sigmaColor : 색 공간에서 필터의 표준 편차(픽셀간 편차보다 커지면 에지라고 판단하는 기준)
- sigmaSpace : 좌표 공간에서 필터의 표준 편차
- dst : 출력 영상. src와 같은 크기, 같은 타입
- borderType : 가장자리 픽셀 처리 방식

In [113]:
import time
start = time.time()
src = cv2.imread('data/ch04/lenna.bmp', cv2.IMREAD_GRAYSCALE)

dst = cv2.bilateralFilter(src, -1, 10, 5)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

print(round(time.time() - start, 2), 'sec')
cv2.waitKey()
cv2.destroyAllWindows()

0.05 sec


# 카툰 필터 카메라